In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm

import sys
sys.path.append('../')
from timelab import MultiColumn, from_data, from_xy_data
from timelab.utils import smash_array

In [2]:
df = pd.read_pickle('processed.pkl')

In [3]:
# df = df[['LNC', 'MAS', 'CSX']]
df = df[['MAS']]

In [4]:
df.head(5)

MAS                                            
                 Open       High        Low      Close     Volume
Date                                                             
2005-12-21  18.991695  19.154659  18.903944  19.085712  1384200.0
2005-12-22  19.085703  19.298810  18.910202  18.929007  1576900.0
2005-12-23  18.954090  18.954090  18.684571  18.816196  2090500.0
2005-12-27  18.929008  18.991687  18.803650  18.847525  1609000.0
2005-12-28  18.853798  18.935281  18.747245  18.803656  2026300.0

In [5]:
# xdata preprocessing

xdata = df.pct_change().dropna()
xdata = MultiColumn(xdata)
xdata.head()

MAS                                        
                Open      High       Low     Close    Volume
Date                                                        
2005-12-22  0.004950  0.007526  0.000331 -0.008211  0.139214
2005-12-23 -0.006896 -0.017862 -0.011932 -0.005960  0.325702
2005-12-27 -0.001323  0.001984  0.006373  0.001665 -0.230328
2005-12-28 -0.003973 -0.002970 -0.003000 -0.002328  0.259354
2005-12-29 -0.002659 -0.000331 -0.001337 -0.000667 -0.240734

In [6]:
# ydata preprocessing

ydata = xdata.copy()

ydata = ydata.channel_apply(lambda x: (x['High'] > 0).astype(float), 'Raise')
ydata = ydata.loc[xdata.index]
ydata = MultiColumn(ydata)
ydata.head()

AttributeError: 'MultiColumn' object has no attribute 'channel_apply'

In [9]:
lookback = 7
horizon = 1
gap = 0

panel = from_xy_data(xdata, ydata, lookback=lookback, horizon=horizon, gap=gap)
panel

/Users/rodrigonader/My Drive/Logspace/Repositories/timeLab/timelab/panel.py:47: UserWarning: Frequency is either not constant or negative.
  warnings.warn("Frequency is either not constant or negative.")
100%|██████████| 3739/3739 [00:00<00:00, 276738.65it/s]

TimePanel
size                            3739
lookback                           7
horizon                            1
gap                                0
num_xunits                         1
num_yunits                         1
num_xchannels                      5
num_ychannels                      1
start            2005-12-22 00:00:00
end              2020-11-10 00:00:00
Name: TimePanel, dtype: object


<TimePanel, size 3739>

In [10]:
train, test = panel.train_test_split(0.2)

In [11]:
pd.Series(panel.y.flatten()).value_counts()

1.0    1887
0.0    1852
dtype: int64

In [12]:
# Should find freq [business days]... Missing calendar?
# panel.find_freq()

In [13]:
# y is 1D so flatten is enough
# y = panel.y.flatten()

In [14]:
train_panels = train.split_units(split_yunits=False)
test_panels = test.split_units(split_yunits=False)

100%|██████████| 1/1 [00:00<00:00, 202.89it/s]


In [18]:
inputs = {}
hidden = {}

for panel_ in tqdm(train_panels):
    name = panel_.xunits[0]

    # Remove unit dimension - it is already being processed separately.
    X = smash_array(panel_.X)

    inputs[name] = tf.keras.Input(shape=(X.shape[1:]), name='input.' + name)

    # Convoluting on the time dimension
    # [lookback] timesteps reduced to 100 nodes
    hidden[name] = tf.keras.layers.SeparableConv1D(10*lookback, lookback, name='conv.' + name, activation=tf.nn.relu)(inputs[name])
    hidden[name] = tf.keras.layers.Flatten(name='flatten.' + name)(hidden[name])
    hidden[name] = tf.keras.layers.Dense(5, activation=tf.nn.relu, name='dense.' + name)(hidden[name])

x = tf.keras.layers.concatenate(list(hidden.values()))
x = tf.keras.layers.Dense(panel.y.shape[1], activation=tf.nn.sigmoid)(x)
outputs = tf.keras.layers.Reshape(panel.y.shape[1:])(x)

model = tf.keras.Model(inputs=list(inputs.values()), outputs=outputs)
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["binary_crossentropy", 'accuracy'])

  0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
x_train = [smash_array(i.X) for i in train_panels]
y_train = train.y

x_test = [smash_array(i.X) for i in test_panels]
y_test = test.y

H = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100)

Epoch 1/100
 8/94 [=>............................] - ETA: 0s - loss: 0.6925 - binary_crossentropy: 0.6925 - accuracy: 0.5039 

2021-09-20 16:54:56.566037: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


94/94 [==============================] - 1s 9ms/step - loss: 0.6929 - binary_crossentropy: 0.6929 - accuracy: 0.5115 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.4960
Epoch 2/100
11/94 [==>...........................] - ETA: 0s - loss: 0.6919 - binary_crossentropy: 0.6919 - accuracy: 0.5369

2021-09-20 16:54:57.406312: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


94/94 [==============================] - 1s 7ms/step - loss: 0.6914 - binary_crossentropy: 0.6914 - accuracy: 0.5266 - val_loss: 0.6929 - val_binary_crossentropy: 0.6929 - val_accuracy: 0.4933
Epoch 3/100
94/94 [==============================] - 1s 7ms/step - loss: 0.6897 - binary_crossentropy: 0.6897 - accuracy: 0.5390 - val_loss: 0.6926 - val_binary_crossentropy: 0.6926 - val_accuracy: 0.4960
Epoch 4/100
94/94 [==============================] - 1s 7ms/step - loss: 0.6869 - binary_crossentropy: 0.6869 - accuracy: 0.5540 - val_loss: 0.6906 - val_binary_crossentropy: 0.6906 - val_accuracy: 0.5227
Epoch 5/100
94/94 [==============================] - 1s 7ms/step - loss: 0.6786 - binary_crossentropy: 0.6786 - accuracy: 0.5864 - val_loss: 0.6820 - val_binary_crossentropy: 0.6820 - val_accuracy: 0.5561
Epoch 6/100
94/94 [==============================] - 1s 7ms/step - loss: 0.6664 - binary_crossentropy: 0.6664 - accuracy: 0.5998 - val_loss: 0.6732 - val_binary_crossentropy: 0.6732 - val_accu

In [20]:
loss = pd.DataFrame({'loss': H.history['loss'], 'val_loss': H.history['val_loss']})
acc = pd.DataFrame({'loss': H.history['accuracy'], 'val_loss': H.history['val_accuracy']})
acc.plot()

In [ ]:
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
%load_ext tensorboard

In [ ]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure(data=[go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])
                ])


fig.update_layout(xaxis_rangeslider_visible=False)

shapes = []
for index in trues.index:
    shapes.append(dict(type='line', yref='paper', y0=0, y1=1, xref='x',
                 x0=index, x1=index))

fig.update_layout(shapes=shapes)

fig.show()

In [ ]:
train, test = stock_panel.split_train_test(0.2)
print(train.X.shape)
print(train.y.shape)

In [ ]:
inputs = tf.keras.Input(shape=(train.X.shape[1:]))

x = tf.keras.layers.Conv1D(2, 2)(inputs)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
outputs = tf.keras.layers.Reshape(train.y.shape[1:])(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["binary_crossentropy", 'AUC'])
model.summary()

In [ ]:
H = model.fit(train.X, train.y, epochs=120, validation_data=(test.X, test.y), verbose=0)

In [ ]:
losses = pd.DataFrame({'train_loss': H.history['loss'], 'val_loss': H.history['val_loss']})
losses.plot()

In [ ]:
aucs = pd.DataFrame({'train_auc': H.history['auc'], 'val_auc': H.history['val_auc']})
aucs.plot()

In [ ]:
preds = model.predict(test.X)

pred_data = pd.DataFrame(index=test.yindex, 
                         data={'ytrue': test.y.flatten(), 
                               'ypred':preds.flatten()}).astype(float)

pred_data['pred'] = pred_data.ypred > 0.9

In [ ]:
pred_data.ypred.plot(kind='scatter')

In [ ]:
thresh = 0.9
pred_data[pred_data.ypred > thresh]

In [ ]:
print(classification_report(pred_data.ytrue, pred_data.ypred > thresh))

In [ ]:
# Option to add multilevel - one per column instead of one above all
# Label: if (tomorrow high - today close) > 0

In [ ]:
# xdata.loc[:, (slice(None), ['Volume'])] = xdata.loc[:, (slice(None), ['Volume'])].shift()

In [ ]:
import plotly.graph_objects as go

def pair_plot(pair, xunit, yunit, xchannels=None, ychannels=None):
    
    
    x = pair.xframe
    y = pair.yframe
    
    x = select(x, units=[xunit], channels=xchannels)
    y = select(y, units=[yunit], channels=ychannels)

    fig = go.Figure()

    for _, channel in enumerate(xchannels):
        c = random.choice(cmap1)
        fig.add_trace(go.Scatter(x=x.index, y=x[channel], name="x_" + channel,
                                 line=dict(width=2, color=c)))
        
    for _, channel in enumerate(ychannels):
        c = random.choice(cmap1)
        fig.add_trace(go.Scatter(x=y.index, y=y[channel], name="y_" + channel,
                                 line=dict(width=2, dash='dot', color=c)))

    fig.update_layout(title='', xaxis_title='Timestamps', yaxis_title='Values')
    fig.show()

In [ ]:
pair_plot(panel.pairs[0], 'MSFT', 'MSFT')

In [ ]:
xdata['QCOM']['High'].plot()

In [ ]:
import re
import requests
import pandas as pd

def get_url_mails(url):
    EMAIL_REGEX=r"[\w\.-]+@[\w\.-]+"

    mails = []
    r=requests.get(url)
    for re_match in re.findall(EMAIL_REGEX, r.text):
        mails.append(re_match)
    return mails

def get_mails(urls):
    mails = {}
    for url in urls:
        mails[url] = list(set(get_url_mails(url)))
    return pd.DataFrame(dict([(k, pd.Series(v)) for k,v in mails.items() ]))

def get_mails_from_csv(csv_filename, output_filename):
    urls = pd.read_csv(csv_filename)['Email']
    get_mails(urls).to_csv(output_filename)

get_mails_from_csv('/Users/rodrigonader/Downloads/pesquisa.csv', 'myout.csv')